In [1]:
!nvidia-smi
!nvcc --version

Thu Nov  3 12:04:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    48W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# nerf-pl 에서 mesh 뽑아내기

원격으로 mesh 를 뽑아내기 위해 mesh extractor 를 colab 으로 구현한다.

이를 위해 최신사양에 맞는 colab에서 돌아가게 수정한 nerf-pl 을 사용한다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
TARGET_PATH = "/content/drive/MyDrive/YonseiUniversity/MetaArt"
%cd $TARGET_PATH

/content/drive/MyDrive/YonseiUniversity/MetaArt


Target path 에 nerf-pl 폴더가 없는 경우 github 에서 clone 한다.

만약에 있는 경우는 아래 단계는 스킵한다.

In [4]:
!git clone https://github.com/maroomir/nerf_pl.git

fatal: destination path 'nerf_pl' already exists and is not an empty directory.


pip 설치를 위해 여기는 반드시 실행해야한다.

In [5]:
%cd nerf_pl
!pip install -r requirements.txt

/content/drive/MyDrive/YonseiUniversity/MetaArt/nerf_pl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Nerf Model 불러오기

기존에 nerf_pl_trainer 에서 훈련한 model을 불러오기 위해 Path를 정의한다.

In [6]:
# Change here #
img_wh = (4032, 3024) # full resolution of the input images
dataset_name = 'llff' # blender or llff (own data)
scene_name = 'fortress' # whatever you want
root_dir = '/content/drive/MyDrive/YonseiUniversity/MetaArt/nerf-pytorch/data/nerf_llff_data/fortress' # the folder containing data (images/ and poses_bounds.npy)
ckpt_path = '/content/drive/MyDrive/YonseiUniversity/MetaArt/nerf_pl/logs/fortress/version_None/checkpoints/epoch=29-step=228840.ckpt' # the model path
###############
N = 128 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -1, 1 # left/right range
ymin, ymax = -1, 1 # forward/backward range
zmin, zmax = -1, 1 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 20. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
###############

## Color 뽑아내기



In [7]:
import os
os.environ['ROOT_DIR'] = root_dir
os.environ['DATASET_NAME'] = dataset_name
os.environ['SCENE_NAME'] = scene_name
os.environ['IMG_SIZE'] = f"{img_wh[0]} {img_wh[1]}"
os.environ['CKPT_PATH'] = ckpt_path
os.environ['N_GRID'] = "512" # final resolution. You can set this number high to preserve more details
os.environ['X_RANGE'] = f"{xmin} {xmax}"
os.environ['Y_RANGE'] = f"{ymin} {ymax}"
os.environ['Z_RANGE'] = f"{zmin} {zmax}"
os.environ['SIGMA_THRESHOLD'] = str(sigma_threshold)
os.environ['OCC_THRESHOLD'] = "0.2" # probably doesn't require tuning. If you find the color is not close
                                    # to real, try to set this number smaller (the effect of this number
                                    # is explained in my youtube video)

!python extract_color_mesh.py \
    --root_dir "$ROOT_DIR" \
    --dataset_name $DATASET_NAME \
    --scene_name $SCENE_NAME \
    --img_wh $IMG_SIZE \
    --ckpt_path $CKPT_PATH \
    --N_grid $N_GRID \
    --x_range $X_RANGE \
    --y_range $Y_RANGE \
    --z_range $Z_RANGE \
    --sigma_threshold $SIGMA_THRESHOLD \
    --occ_threshold $OCC_THRESHOLD

tcmalloc: large alloc 1073741824 bytes == 0x5f298000 @  0x7f39422db1e7 0x7f38d1b920ce 0x7f38d1bec726 0x7f38d1bdf475 0x7f38d1c7fe51 0x4d22a6 0x51041f 0x5b4ee6 0x58ff2e 0x50c4fc 0x5b575e 0x4bad99 0x7f38d1bd3944 0x58f67f 0x50ff13 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7f3941ed8c87 0x5b636a
tcmalloc: large alloc 3221225472 bytes == 0x11f298000 @  0x7f39422db1e7 0x7f38d1b920ce 0x7f38d1be8cf5 0x7f38d1c9186d 0x7f38d1c9217f 0x7f38d1c922d0 0x4ba72b 0x7f38d1bd3944 0x58f67f 0x50ff13 0x5b4ee6 0x58ff2e 0x50d482 0x5b575e 0x4bad99 0x7f38d1bd3944 0x58f67f 0x50ff13 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7f3941ed8c87 0x5b636a
tcmalloc: large alloc 1610612736 bytes == 0x5f298000 @  0x7f39422bdb6b 0x7f39422dd379 0x7f38d287ad57 0x7f38d2868bc3 0x7f38fc77287f 0x7f38fc772aca 0x7f38fc772b2f 0x7f38fcc85e49 0x7f38fd4f7cb0 0x7f38fd2433ee 0x7f38fd4d0e4e 0x7f38fd2832c2 0x7f38fcc802f1 0x7f38fd64a64b 0x7f38fcfcdf